In [1]:
import pandas as pd
from datetime import datetime, timedelta
import json

In [2]:
!mkdir -p sailboat_attempts
!mkdir -p microsailboat_attempts

## Special cases

### Aberystwyth attempt 2

In [3]:
aber2 = pd.read_csv('aber-own-gps/AberSailbot-AreaScan-Attempt2.csv')

# Move datetime first
aber2 = aber2[['datetime', 'latitude', 'longitude']]

# Select time for attempt - timestamps are UTC, but without 'Z'
inwindow = (('2018-08-29T14:29:00' < aber2.datetime) & (aber2.datetime < '2018-08-29T15:14:00'))
aber2 = aber2[inwindow]

# Strip off microseconds, add Z for UTC timezone
aber2.datetime = aber2.datetime.map(str).str[:len('2018-08-29T14:29:00')] + 'Z'

aber2.to_csv('microsailboat_attempts/Aberystwyth2.csv', index=False)

### Plymouth large boat 1

In [4]:
datetime.strptime('26/08/2018 13:22:12+0000', '%d/%m/%Y %H:%M:%S%z')

datetime.datetime(2018, 8, 26, 13, 22, 12, tzinfo=datetime.timezone.utc)

In [5]:
plym_big1 = pd.read_csv('plymouth-own-gps/pb_first_attempt.csv')
plym_big1.columns = ['nix', 'datetime', 'latitude', 'longitude']
plym_big1 = plym_big1[['datetime', 'latitude', 'longitude']]
plym_big1.datetime = [datetime.strptime(s + '+0000', '%d/%m/%Y %H:%M:%S%z') for s in plym_big1.datetime]

# We think this is already trimmed to only the scorable data
plym_big1.to_csv('sailboat_attempts/Plymouth1.csv', index=False)

### Plymouth small boat 2

In [6]:
plym_small2 = pd.read_csv('plymouth-own-gps/ps_second_attempt.csv')
plym_small2.columns = ['nix', 'datetime', 'latitude', 'longitude']
plym_small2 = plym_small2[['datetime', 'latitude', 'longitude']]
plym_small2.datetime = [datetime.strptime(s + '+0000', '%d/%m/%Y %H:%M:%S%z') for s in plym_small2.datetime]

# This should already be trimmed to size
plym_small2.to_csv('microsailboat_attempts/Plymouth2.csv', index=False)

## Small boats

In [7]:
import json
with open('attempts.json') as f:
    attempts_times = json.load(f)['teams']

In [8]:
# team, attempt no, source file
micro_attempts = [
    ("Plymouth (Micro-Sailboat)", 1, "Plymouth1.csv"),
    ("ZMART", 1, "ZMART1.csv"),
    ("ZMART", 2, "ZMART2.csv"),
    # TODO: get Southampton first attempt from team
    ("Southampton", 2, "Southampton2.csv"),
    ("AberSailbot", 1, "Aberystwyth1.csv"),
]

def process_attempt(teamname, attemptno, sourcefile, to_folder):
    print(teamname, attemptno)
    track_data = pd.read_csv(sourcefile, parse_dates=[0])
    attempt_times = attempts_times[teamname][attemptno - 1]
    start_t = datetime.strptime(attempt_times['start'], '%Y-%m-%dT%H:%M:%S%z')
    end_t  = datetime.strptime(attempt_times['end'], '%Y-%m-%dT%H:%M:%S%z')
    in_attempt = (track_data.datetime >= start_t) & (track_data.datetime <= end_t)
    print(in_attempt.sum(), "points in attempt")
    
    # Now subtract interruptions
    intrpts_n = 0
    in_intrpt = track_data.datetime < '2000-01-01'  # Start with all False, add interruptions
    for intrpt in attempt_times['interruptions']:
        int_start_t = datetime.strptime(intrpt['from'], '%Y-%m-%dT%H:%M:%S%z')
        int_end_t = datetime.strptime(intrpt['to'], '%Y-%m-%dT%H:%M:%S%z')
        if (int_end_t - int_start_t) <= timedelta(minutes=2):
            # Ignore interruptions of 2 minutes or less
            continue
        
        intrpts_n += 1
        in_this_intrpt = (track_data.datetime > int_start_t) & (track_data.datetime < int_end_t)
        in_intrpt |= in_this_intrpt
    
    print('Subtracting', intrpts_n, 'interruptions')
    track_data = track_data[in_attempt & ~in_intrpt]
    print(len(track_data), "points remaining")
    filename = '{}/{}{}.csv'.format(to_folder, teamname.split()[0], attemptno)
    track_data.to_csv(filename)
    print("Written", filename)
    print()

for teamname, attemptno, sourcefile in micro_attempts:
    process_attempt(teamname, attemptno, sourcefile, 'microsailboat_attempts')

Plymouth (Micro-Sailboat) 1
2616 points in attempt
Subtracting 0 interruptions
2616 points remaining
Written microsailboat_attempts/Plymouth1.csv

ZMART 1
522 points in attempt
Subtracting 0 interruptions
522 points remaining
Written microsailboat_attempts/ZMART1.csv

ZMART 2
2242 points in attempt
Subtracting 1 interruptions
2070 points remaining
Written microsailboat_attempts/ZMART2.csv

Southampton 2
2707 points in attempt
Subtracting 0 interruptions
2707 points remaining
Written microsailboat_attempts/Southampton2.csv

AberSailbot 1
218 points in attempt
Subtracting 0 interruptions
218 points remaining
Written microsailboat_attempts/AberSailbot1.csv



In [9]:
sailboat_attempts = [
    ("ENSTA", 1, "ENSTA1.csv"),
    ("SJTU", 1, "Shanghai1.csv"),
    ("SJTU", 2, "Shanghai1.csv"),
    ("Åland", 1, "Åland1.csv"),
]

for teamname, attemptno, sourcefile in sailboat_attempts:
    process_attempt(teamname, attemptno, sourcefile, 'sailboat_attempts')

ENSTA 1
2488 points in attempt
Subtracting 0 interruptions
2488 points remaining
Written sailboat_attempts/ENSTA1.csv

SJTU 1
1957 points in attempt
Subtracting 0 interruptions
1957 points remaining
Written sailboat_attempts/SJTU1.csv

SJTU 2
1785 points in attempt
Subtracting 0 interruptions
1785 points remaining
Written sailboat_attempts/SJTU2.csv

Åland 1
1092 points in attempt
Subtracting 1 interruptions
803 points remaining
Written sailboat_attempts/Åland1.csv

